In [1]:
# bord of the game
from math import *
import random

In [2]:
# Tic Tac Toe Game State
class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass
class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move]     = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
    
    def Gameover(self):
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z] and not self.board[z]==0 :
                return True
        return False
    
    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s

In [ ]:
class my_node():
    
    def __init__(self,parent,state):
        
        # relationship with other trees
        self.parent = parent
        self.childrens    = []
        
        #current state of the board
        self.current_state = state
        self.avaiable_moves = state.GetMoves()
        
        # calculate the stats
        self.number_visit = 0
        self.number_wins  = 0 
    
    def Q(self): return 
    
    def select_a_node(self):
        pass
    
    def AddChild(self):
        pass
    
    def backprop(self):
        pass
        

In [71]:
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins   += result

    def __repr__(self):
        return "[M:" + str(self.move) + " Wins/Visits:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent=1):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes: s += c.TreeToString(indent+1)
        return s
    def IndentString(self,indent=1):
        s = "\n"
        for i in range (1,indent+1): s += "| "
        return s
    def ChildrenToString(self):
        s = ""
        for c in self.childNodes: s += str(c) + "\n"
        return s

def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node  = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if (verbose): print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
    
""" Play a sample game between two UCT players where each player gets a different number 
    of UCT iterations (= simulations = tree nodes).
"""
state = OXOState() # uncomment to play OXO
while (state.GetMoves() != []):
    if state.Gameover(): break
    print(str(state))
    if state.playerJustMoved == 1:
        m = UCT(rootstate = state, itermax = 1000, verbose = False) # play with values for itermax and verbose = True
    else:
        m = UCT(rootstate = state, itermax = 2,   verbose = False)
    print("Best Move: " + str(m) + "\n")
    state.DoMove(m)
    

# print the 
if state.GetResult(state.playerJustMoved) == 1.0:  print("Player " + str(state.playerJustMoved)     + " wins!")
elif state.GetResult(state.playerJustMoved) == 0.0:print("Player " + str(3 - state.playerJustMoved) + " wins!")
else: print("Nobody wins!")

...
...
...

[M:6 Wins/Visits:1.0/1 U:[0, 1, 2, 3, 4, 5, 7, 8]]
[M:1 Wins/Visits:1.0/1 U:[0, 2, 3, 4, 5, 6, 7, 8]]

Best Move: 1

.X.
...
...

[M:8 Wins/Visits:10.5/58 U:[]]
[M:6 Wins/Visits:50.5/142 U:[]]
[M:3 Wins/Visits:23.5/87 U:[]]
[M:2 Wins/Visits:24.5/89 U:[]]
[M:5 Wins/Visits:30.5/102 U:[]]
[M:4 Wins/Visits:77.5/193 U:[]]
[M:7 Wins/Visits:13.0/64 U:[]]
[M:0 Wins/Visits:116.5/265 U:[]]

Best Move: 0

OX.
...
...

[M:6 Wins/Visits:1.0/1 U:[2, 3, 4, 5, 7, 8]]
[M:3 Wins/Visits:1.0/1 U:[2, 4, 5, 6, 7, 8]]

Best Move: 3

OX.
X..
...

[M:4 Wins/Visits:399.5/658 U:[]]
[M:6 Wins/Visits:22.0/71 U:[]]
[M:5 Wins/Visits:20.0/68 U:[]]
[M:8 Wins/Visits:21.0/69 U:[]]
[M:2 Wins/Visits:15.0/58 U:[]]
[M:7 Wins/Visits:24.5/76 U:[]]

Best Move: 4

OX.
XO.
...

[M:6 Wins/Visits:0.0/1 U:[2, 5, 7, 8]]
[M:8 Wins/Visits:0.5/1 U:[2, 5, 6, 7]]

Best Move: 8

OX.
XO.
..X

[M:7 Wins/Visits:65.5/152 U:[]]
[M:2 Wins/Visits:188.5/352 U:[]]
[M:6 Wins/Visits:183.0/344 U:[]]
[M:5 Wins/Visits:65.5/152 U:[]]

Best 

In [28]:
state = OXOState()
print(state.GetMoves())
print(state.playerJustMoved)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
2


In [27]:
m

8

In [60]:
my_state = OXOState() # uncomment to play OXO
print(my_state.GetMoves())
# while (state.GetMoves() != []):
#     print(str(state))
#     if state.playerJustMoved == 1:
#         m = UCT(rootstate = state, itermax = 1000, verbose = False)

my_node       = Node(state=state)
my_state_copy = my_state.Clone()
print(state)
print(dir(my_node))
print(my_node.TreeToString())
print(my_node.untriedMoves)

# if we have tried every moves. 
while my_node.untriedMoves == [] and my_node.childNodes != []: # node is fully expanded and non-terminal
    my_node = my_node.UCTSelectChild()
    my_state_copy.DoMove(my_node.move)
    
# Expand
if my_node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
    m = random.choice(my_node.untriedMoves) 
    my_state_copy.DoMove(m)
    my_node = my_node.AddChild(m,my_state_copy) # add child and descend tree
    
print('---')
print(my_node )
print(my_node.untriedMoves)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
...
...
...

['AddChild', 'ChildrenToString', 'IndentString', 'TreeToString', 'UCTSelectChild', 'Update', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'childNodes', 'move', 'parentNode', 'playerJustMoved', 'untriedMoves', 'visits', 'wins']

| [M:None W/V:0/0 U:[0, 1, 2, 3, 4, 5, 6, 7, 8]]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
---
[M:4 W/V:0/0 U:[0, 1, 2, 3, 5, 6, 7, 8]]
[0, 1, 2, 3, 5, 6, 7, 8]


In [61]:
# main function for the Monte Carlo Tree Search 
def monte_carlo_tree_search(root): 
      
    while resources_left(time, computational power): 
        leaf = traverse(root)  
        simulation_result = rollout(leaf) 
        backpropagate(leaf, simulation_result) 
          
    return best_child(root) 
  
# function for node traversal 
def traverse(node): 
    # here we are just going down the tree
    while fully_expanded(node): 
        node = best_uct(node) 
          
    # in case no children are present / node is terminal  
    return pick_univisted(node.children) or node  
  
# function for the result of the simulation 
def rollout(node): 
    # until not terminal we are going to roll out - > play out
    while non_terminal(node): 
        node = rollout_policy(node) 
    return result(node)  
  
# function for randomly selecting a child node 
def rollout_policy(node): 
    return pick_random(node.children) 
  
# function for backpropagation 
# this seems to be a bell man type of equation
def backpropagate(node, result): 
    if is_root(node) return
    node.stats = update_stats(node, result)  
    backpropagate(node.parent) 
  
# function for selecting the best child 
# node with highest number of visits 
def best_child(node):  pick child with highest number of visits 

SyntaxError: invalid syntax (<ipython-input-61-f79107858af3>, line 4)

# Reference 
1. jbradberry/ultimate_tictactoe. (2019). GitHub. Retrieved 26 March 2019, from https://github.com/jbradberry/ultimate_tictactoe/blob/master/t3/board.py
2. https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/
3. Monte Carlo Tree Search - Python Code. (2019). Mcts.ai. Retrieved 26 March 2019, from http://mcts.ai/code/python.html
